In [14]:
import numpy as np
import csv
import random
import pandas as pd

class matrix:
    def __init__(self, filename=None):
      # Initialize the array_2d as None
      self.array_2d = None
      # If a filename is provided, load data from the CSV
      if filename:
          self.load_from_csv(filename)

    def load_from_csv(self, filename):
      # Load the CSV using pandas
      data = pd.read_csv(filename, header=None)  # Read CSV without assuming headers
      # Convert the DataFrame into a NumPy array and assign it to array_2d
      self.array_2d = data.values  # Extract the numpy array from the pandas DataFrame


    def standardise(self):
      k1=self.array_2d
      n=len(k1)
      m=len(k1[0])
      for i in range(n):
        for j in range(m):
          k1[i][j]=(k1[i][j]-np.mean(k1[:, j]))/(np.max(k1[:, j])-np.min(k1[:, j]))
      self.array_2d = k1

    def get_distance(self, other_matrix, row_i):
        # Extract the specified row from the current matrix
        row = self.array_2d[row_i, :]
        # Compute the difference between this row and each row in other_matrix
        diffs = other_matrix.array_2d - row
        # Compute the squared differences
        sq_diffs = diffs ** 2
        # Sum the squared differences across columns to get squared Euclidean distances
        sum_sq_diffs = np.sum(sq_diffs, axis=1)
        # Take the square root to get Euclidean distances
        distances = np.sqrt(sum_sq_diffs)
        # Reshape the distances to be a column vector
        return matrix_from_array(distances.reshape(-1, 1))


    def get_weighted_distance(self, other_matrix, weights, row_i):
        # Extract the specified row from the current matrix
        row = self.array_2d[row_i, :]
        # Flatten the weights matrix to a 1D array
        w = weights.array_2d.flatten()
        # Compute the difference between this row and each row in other_matrix
        diffs = other_matrix.array_2d - row
        # Compute the squared differences
        sq_diffs = diffs ** 2
        # Apply the weights to the squared differences
        weighted_sq_diffs = sq_diffs * w
        # Sum the weighted squared differences across columns
        sum_weighted_sq_diffs = np.sum(weighted_sq_diffs, axis=1)
        # Take the square root to get Weighted Euclidean distances
        distances = np.sqrt(sum_weighted_sq_diffs)
        # Reshape the distances to be a column vector
        return matrix_from_array(distances.reshape(-1, 1))

    def get_count_frequency(self):
      if len(self.array_2d[0])==1:
        # Flatten the matrix to a 1D array
        flattened_matrix = (self.array_2d).flatten()
        # Get unique elements and their frequencies using np.unique
        unique_elements, counts = np.unique(flattened_matrix, return_counts=True)
        # Create a dictionary mapping each unique element to its frequency
        frequency_dict = {unique_elements[i]: counts[i] for i in range(len(unique_elements))}
        return frequency_dict
      else:
        return 0


def matrix_from_array(array):
    m = matrix()
    m.array_2d = array
    return m

def get_initial_weights(m):

    # Generate m random numbers between 0 and 1
    random_values = np.random.rand(m)
    # Normalize the weights so that their sum is 1
    random_values /= np.sum(random_values)
    # Reshape to a 1xM matrix
    array = random_values.reshape(1, -1)
    return matrix_from_array(array)

def get_centroids(data, S, K):

    m = data.array_2d.shape[1]  # Number of features
    centroids_array = np.zeros((K, m))  # Initialize centroids array

    for k in range(1, K+1):
        # Find indices of data points assigned to cluster k
        indices = np.where(S.array_2d.flatten() == k)[0]
        if len(indices) > 0:
            # Compute the mean of the assigned data points for each feature
            centroids_array[k-1, :] = np.mean(data.array_2d[indices, :], axis=0)
        else:
            # If no points are assigned to cluster k, reinitialize the centroid randomly
            centroids_array[k-1, :] = data.array_2d[np.random.randint(0, data.array_2d.shape[0]), :]

    return matrix_from_array(centroids_array)

def get_separation_within(data, centroids, S, K):

    n, m = data.array_2d.shape  # Number of data points and features
    a = np.zeros(m)  # Initialize separation within array

    for j in range(m):
        for k in range(1, K+1):
            # Find indices of data points assigned to cluster k
            indices = np.where(S.array_2d.flatten() == k)[0]
            if len(indices) > 0:
                # Calculate the absolute distance between each data point and the centroid for feature j
                distances = np.abs(data.array_2d[indices, j] - centroids.array_2d[k-1, j])**2
                                # Sum the distances for feature j
                a[j] += np.sum(distances)

    # Reshape to a 1xM matrix
    return matrix_from_array(a.reshape(1, -1))

def get_separation_between(data, centroids, S, K):

    m = data.array_2d.shape[1]  # Number of features
    b = np.zeros(m)  # Initialize separation between array
    # Compute the overall mean for each feature
    DDj = np.mean(data.array_2d, axis=0)

    for k in range(1, K+1):
        # Count the number of data points in cluster k
        Nk = np.sum(S.array_2d.flatten() == k)
        for j in range(m):
            # Calculate the absolute distance between the centroid and the overall mean for feature j
            distance = np.abs(centroids.array_2d[k-1, j] - DDj[j])**2
            # Multiply by the number of data points in the cluster and add to separation between
            b[j] += Nk * distance

    # Reshape to a 1xM matrix
    return matrix_from_array(b.reshape(1, -1))

def get_groups(data, K):

    n, m = data.array_2d.shape  # Number of data points and features

    # Step 2: Initialize weights
    weights = get_initial_weights(m)

    # Step 5: Randomly select K unique data points as initial centroids
    random_indices = random.sample(range(n), K)
    centroids_array = data.array_2d[random_indices, :]
    centroids = matrix_from_array(centroids_array)

    # Step 4: Initialize matrix S with zeros
    S = matrix_from_array(np.zeros((n, 1), dtype=int))

    iteration = 0  # To keep track of iterations

    while True:
        iteration += 1
        S_prev = S.array_2d.copy()  # Keep a copy of the previous assignments

        # Step 7: Assign each data point to the nearest centroid
        for i in range(n):
            # Calculate weighted distances between data point i and all centroids
            distances = data.get_weighted_distance(centroids, weights, i).array_2d.flatten()
            # Find the index of the nearest centroid (1-based indexing)
            nearest = np.argmin(distances) + 1
            # Assign the data point to the nearest cluster
            S.array_2d[i, 0] = nearest

        # Step 8: Check for convergence (if assignments do not change)
        if np.array_equal(S.array_2d, S_prev):
            # print(f"Converged after {iteration} iterations for K={K}")
            break

        # Step 9: Update centroids based on current assignments
        centroids = get_centroids(data, S, K)

        # Step 10: Update weights based on current assignments and centroids
        weights = get_new_weights(data, centroids, weights, S, K)

    return S

def get_new_weights(data, centroids, weights_old, S, K):

    # Calculate separation within clusters (A)
    a = get_separation_within(data, centroids, S, K).array_2d.flatten()
    # Calculate separation between clusters (B)
    b = get_separation_between(data, centroids, S, K).array_2d.flatten()

    # Calculate Bj/Aj for each feature j, handling division by zero
    ratio = np.divide(b, a, out=np.zeros_like(b), where=a!=0)

    # Calculate the sum of (Bj/Aj) across all features
    sum_ratio = np.sum(ratio)

    if sum_ratio == 0:
        sum_ratio = 1  # To prevent division by zero

    # Calculate the new weights using the provided formula
    w_new = 0.5 * (weights_old.array_2d.flatten() + (ratio / sum_ratio))

    # Reshape to a 1xM matrix
    return matrix_from_array(w_new.reshape(1, -1))

def run_test():
    m = matrix('Data.csv')
    # m = m.array_2d
    for k in range (2,11):
        for i in range(20):
            S = get_groups (m, k)
            print(str(k)+'='+str(S.get_count_frequency()))

run_test()


2={1: 55, 2: 123}
2={1: 122, 2: 56}
2={1: 122, 2: 56}
2={1: 122, 2: 56}
2={1: 122, 2: 56}
2={1: 122, 2: 56}
2={1: 55, 2: 123}
2={1: 122, 2: 56}
2={1: 55, 2: 123}
2={1: 56, 2: 122}
2={1: 56, 2: 122}
2={1: 56, 2: 122}
2={1: 122, 2: 56}
2={1: 56, 2: 122}
2={1: 122, 2: 56}
2={1: 55, 2: 123}
2={1: 56, 2: 122}
2={1: 122, 2: 56}
2={1: 55, 2: 123}
2={1: 55, 2: 123}
3={1: 47, 2: 62, 3: 69}
3={1: 47, 2: 62, 3: 69}
3={1: 47, 2: 62, 3: 69}
3={1: 62, 2: 69, 3: 47}
3={1: 47, 2: 62, 3: 69}
3={1: 69, 2: 47, 3: 62}
3={1: 47, 2: 62, 3: 69}
3={1: 69, 2: 62, 3: 47}
3={1: 27, 2: 50, 3: 101}
3={1: 69, 2: 47, 3: 62}
3={1: 27, 2: 50, 3: 101}
3={1: 62, 2: 69, 3: 47}
3={1: 69, 2: 47, 3: 62}
3={1: 62, 2: 69, 3: 47}
3={1: 47, 2: 62, 3: 69}
3={1: 69, 2: 62, 3: 47}
3={1: 62, 2: 69, 3: 47}
3={1: 47, 2: 62, 3: 69}
3={1: 47, 2: 62, 3: 69}
3={1: 62, 2: 69, 3: 47}
4={1: 59, 2: 57, 3: 23, 4: 39}
4={1: 23, 2: 39, 3: 59, 4: 57}
4={1: 57, 2: 23, 3: 59, 4: 39}
4={1: 23, 2: 59, 3: 39, 4: 57}
4={1: 39, 2: 23, 3: 57, 4: 59}
4={